## Imports

In [9]:
import requests
from bs4 import BeautifulSoup
import os
from astropy.io import fits
from astropy.time import Time
import matplotlib.pyplot as plt
import numpy as np

## Parameters 

In [35]:
name = "SN2023cv"

## Downloading FITS Files

In [36]:
base_url = "https://spt3g.ncsa.illinois.edu/files/jobs/e508c213baed4cfaa4d764685ed3e771/out/SN2023cv/"

download_folder = f"/Volumes/Memorex USB/{name}/fits_files"
os.makedirs(download_folder, exist_ok=True)


response = requests.get(base_url)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")

file_links = [
    base_url + link["href"]
    for link in soup.find_all("a", href=True)
    if not link["href"].startswith("?") and not link["href"].endswith("/")
]

print(f"Found {len(file_links)} files to download.")

for idx, file_url in enumerate(file_links, start=1):
    file_name = os.path.basename(file_url)
    save_path = os.path.join(download_folder, file_name)

    try:
        with requests.get(file_url, stream=True) as r:
            r.raise_for_status()
            with open(save_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"({idx}/{len(file_links)}) Downloaded: {file_name}")
    except Exception as e:
        print(f"Error downloading {file_name}: {e}")

Found 138 files to download.
(1/138) Downloaded: SN2023cv_150GHz_231400425_psth.fits
(2/138) Downloaded: SN2023cv_150GHz_231407529_psth.fits
(3/138) Downloaded: SN2023cv_150GHz_231414634_psth.fits
(4/138) Downloaded: SN2023cv_150GHz_231422500_psth.fits
(5/138) Downloaded: SN2023cv_150GHz_231429684_psth.fits
(6/138) Downloaded: SN2023cv_150GHz_231436791_psth.fits
(7/138) Downloaded: SN2023cv_150GHz_231443896_psth.fits
(8/138) Downloaded: SN2023cv_150GHz_231722991_psth.fits
(9/138) Downloaded: SN2023cv_150GHz_231730095_psth.fits
(10/138) Downloaded: SN2023cv_150GHz_231737199_psth.fits
(11/138) Downloaded: SN2023cv_150GHz_231745061_psth.fits
(12/138) Downloaded: SN2023cv_150GHz_231752167_psth.fits
(13/138) Downloaded: SN2023cv_150GHz_231759273_psth.fits
(14/138) Downloaded: SN2023cv_150GHz_231766378_psth.fits
(15/138) Downloaded: SN2023cv_150GHz_233740805_psth.fits
(16/138) Downloaded: SN2023cv_150GHz_233747909_psth.fits
(17/138) Downloaded: SN2023cv_150GHz_233755012_psth.fits
(18/138) Do

## Inputs and Outputs

In [37]:
input_folder = download_folder
output_csv = "/Users/Djslime07/SPT_VSCode/CSV's/obs_values.csv"

## FITS Plotter

In [38]:
for fits_file in os.listdir(input_folder):
    if fits_file.endswith(".fits"):
        fits_path = os.path.join(input_folder, fits_file)

        if fits_file.startswith("._"):
            continue

        try:
            with fits.open(fits_path, ignore_missing_simple=True) as hdul:
                data = hdul[0].data
                header = hdul[0].header
                date_beg = header.get('DATE-BEG', 'Unknown')
                date_end = header.get('DATE-END', 'Unknown')
                object_name = header.get('OBJECT', 'Unknown')
                band = header.get('BAND', 'Unknown')
                print(f"Processing {fits_file} | DATE-BEG: {date_beg} | BAND: {band}")

                try:
                    mjd = Time(date_beg).mjd
                except Exception as e:
                    print(f"Skipping {fits_file}: Invalid DATE-BEG format. Error: {e}")
                    continue

                if data is None:
                    print(f"Skipping {fits_file}: No image data found.")
                    continue

                data = np.nan_to_num(data)
                if np.max(data) == 0:
                    print(f"Skipping {fits_file}: Peak flux is zero.")
                    continue


                if "fltd" in fits_file:
                    suffix = "Filtered"
                elif "psth" in fits_file:
                    suffix = "Passthrough"
                else:
                    print(f"Skipping {fits_file}: Unknown file type.")
                    continue

                peak_flux = np.max(data)
                rms_temp = np.std(data)
                keep = np.abs(data) <= 3 * rms_temp
                rms = np.std(data[keep])
                vmin, vmax = np.percentile(data, [1, 99])


                band_output_folder = f"/Volumes/Memorex USB/{name}/PNG_{name}_{band}"
                os.makedirs(band_output_folder, exist_ok=True)

                plt.figure()
                plt.imshow(data, origin='lower', vmin=vmin, vmax=vmax)
                plt.colorbar(label='Flux [mK]')
                plt.title(
                    f"{object_name} ({band}) MJD: {mjd:.5f} | Peak Flux: {peak_flux:.4} mK | Flux RMS: {rms:.4f} mK",
                    fontsize=9.5
                )
                png_filename = os.path.join(band_output_folder, f"{mjd:.5f}_{suffix}.png")
                plt.savefig(png_filename)
                plt.close()


                with open(output_csv, "a") as f:
                    print(f"{object_name},{date_beg},{date_end},{mjd},{peak_flux},{rms},{band}", file=f)

            print(f"Converted: {fits_file} -> {png_filename}")
        except Exception as e:
            print(f"Error processing {fits_file}: {e}")

Processing SN2023cv_150GHz_231400425_psth.fits | DATE-BEG: 2024-05-02T05:53:46.000 | BAND: 150GHz
Converted: SN2023cv_150GHz_231400425_psth.fits -> /Volumes/Memorex USB/SN2023cv/PNG_SN2023cv_150GHz/60432.24567_Passthrough.png
Processing SN2023cv_150GHz_231407529_psth.fits | DATE-BEG: 2024-05-02T07:52:10.000 | BAND: 150GHz
Converted: SN2023cv_150GHz_231407529_psth.fits -> /Volumes/Memorex USB/SN2023cv/PNG_SN2023cv_150GHz/60432.32789_Passthrough.png
Processing SN2023cv_150GHz_231414634_psth.fits | DATE-BEG: 2024-05-02T09:50:35.000 | BAND: 150GHz
Converted: SN2023cv_150GHz_231414634_psth.fits -> /Volumes/Memorex USB/SN2023cv/PNG_SN2023cv_150GHz/60432.41013_Passthrough.png
Processing SN2023cv_150GHz_231422500_psth.fits | DATE-BEG: 2024-05-02T12:01:41.000 | BAND: 150GHz
Converted: SN2023cv_150GHz_231422500_psth.fits -> /Volumes/Memorex USB/SN2023cv/PNG_SN2023cv_150GHz/60432.50117_Passthrough.png
Processing SN2023cv_150GHz_231429684_psth.fits | DATE-BEG: 2024-05-02T14:01:25.000 | BAND: 150GH

## Data Extraction

In [6]:
for fits_file in os.listdir(input_folder):
    if fits_file.endswith(".fits"):
        fits_path = os.path.join(input_folder, fits_file)

        if fits_file.startswith("._"):
            continue

        try:
            with fits.open(fits_path, ignore_missing_simple=True) as hdul:
                data = hdul[0].data
                header = hdul[0].header
                date_beg = header.get('DATE-BEG', 'Unknown')
                date_end = header.get('DATE-END', 'Unknown')
                object_name = header.get('OBJECT', 'Unknown')
                band = header.get('BAND', 'Unknown')
                print(f"Date:{date_beg}")

                try:
                    mjd = Time(date_beg).mjd
                except Exception as e:
                    print(f"Skipping {fits_file}: Invalid DATE-BEG format. Error: {e}")
                    continue

                if data is None:
                    print(f"Skipping {fits_file}: No image data found.")
                    continue

                data = np.nan_to_num(data)
                if np.max(data) == 0:
                    print(f"Skipping {fits_file}: Peak flux is zero.")
                    continue

                if "fltd" in fits_file:
                    suffix = "Filtered"
                elif "psth" in fits_file:
                    suffix = "Passthrough"
                else:
                    print(f"Skipping {fits_file}: Unknown file type.")
                    continue

                peak_flux = np.max(data)
                rms_temp = np.std(data)
                keep = np.abs(data) <= 3 * rms_temp
                rms = np.std(data[keep])

                with open(output_csv, "a") as f:
                    print(f"{date_beg}, {date_end}, {mjd} ,{peak_flux}, {rms}, {band}", file=f)
        except Exception as e:
            print(f"Error processing {fits_file}: {e}")            

Error processing SN2019esa_220GHz_217786563_fltd.fits: Empty or corrupt FITS file
Date:2023-11-26T18:36:03.000
Error processing SN2019esa_220GHz_218515294_fltd.fits: Empty or corrupt FITS file
Error processing SN2019esa_220GHz_218523693_fltd.fits: Empty or corrupt FITS file
Error processing SN2019esa_220GHz_218532094_fltd.fits: Empty or corrupt FITS file
Error processing SN2019esa_220GHz_218541259_fltd.fits: buffer is too small for requested array
Date:2023-12-05T12:14:20.000
Error processing SN2019esa_220GHz_218558058_fltd.fits: buffer is too small for requested array
Error processing SN2019esa_220GHz_218939212_fltd.fits: Empty or corrupt FITS file
Error processing SN2019esa_220GHz_218956010_fltd.fits: Empty or corrupt FITS file
Error processing SN2019esa_220GHz_218965177_fltd.fits: Empty or corrupt FITS file
Error processing SN2019esa_220GHz_218973576_fltd.fits: Empty or corrupt FITS file
Error processing SN2019esa_220GHz_218981975_fltd.fits: Empty or corrupt FITS file
Error processi

        Use textwrap.indent() instead. [astropy.io.fits.hdu.hdulist]
    Header size is not multiple of 2880: 8192
There may be extra bytes after the last HDU or the file is corrupted. [astropy.io.fits.hdu.hdulist]
